### Установка пакетов

In [1]:
!pip install pyspark

### Импорт библиотек

In [2]:
import json
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
import urllib.request

### Создание каталога data для хранения данных

In [3]:
if not os.path.exists('data/input'):
  os.makedirs('data/input')

if not os.path.exists('data/output'):
  os.makedirs('data/output')

urllib.request.urlretrieve('https://raw.githubusercontent.com/Losyash/sepbd_i_hw/refs/heads/master/lab_5/data/Invoice-set1.json', 'data/input/Invoice-set1.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/Losyash/sepbd_i_hw/refs/heads/master/lab_5/data/Invoice-set2.json', 'data/input/Invoice-set2.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/Losyash/sepbd_i_hw/refs/heads/master/lab_5/data/Invoice-set3.json', 'data/input/Invoice-set3.json')

('data/input/Invoice-set3.json', <http.client.HTTPMessage at 0x7c94073a7580>)

### Создание экземпляра SparkSession

In [4]:
spark = SparkSession\
  .builder \
  .appName('Homework_5_lab3') \
  .config('spark.sql.streaming.schemaInference', 'true') \
  .getOrCreate()

## Structured Streaming из набора файлов

### Задание 1. Подготовить данные

In [5]:
df = spark.readStream \
  .format('json') \
  .option('path', 'data/input') \
  .option('maxFilesPerTrigger', 1) \
  .load()

In [6]:
df_flat_1 = df.selectExpr(
  'InvoiceNumber',
  'CreatedTime',
  'StoreID',
  'PosID',
  'CashierID',
  'CustomerType',
  'CustomerCardNo',
  'TotalAmount',
  'NumberOfItems',
  'PaymentMethod',
  'TaxableAmount',
  'explode(InvoiceLineItems) as Item'
)

### Задание 2. Написать преобразование исходного датафрейма, для получения плоской (flat) структуры.

In [7]:
df_flat_2 = df_flat_1 \
  .withColumn('ItemCode', expr('Item.ItemCode')) \
  .withColumn('temDescription', expr('Item.ItemDescription')) \
  .withColumn('ItemPrice', expr('Item.ItemPrice')) \
  .withColumn('ItemQty', expr('Item.ItemQty')) \
  .withColumn('TotalValue', expr('Item.TotalValue')) \
  .drop('Item')

### Задание 3-4. Открыть на основе исходных json файлов потоковый датафрейм и сохранить получаемые каждые X секунд исходные файлы в json.

In [8]:
stream = df_flat_2.writeStream \
  .format('json') \
  .option('path', 'data/output') \
  .option('checkpointLocation', 'data/check-dir') \
  .trigger(processingTime='30 seconds') \
  .start()

In [9]:
#spark.stop()